In [52]:
from faker import Faker
import google.auth
from google.cloud.sql.connector import Connector
from google.auth.transport.requests import Request
import sqlalchemy
import random

In [54]:
# initialize parameters
# INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
# print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "rustom"
DB_PASS = "hasbulla"
DB_NAME = "family-hub-database"

INSTANCE_CONNECTION_NAME = 'familyhub-366019:us-central1:family-hub-dev'
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"


In [89]:
# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

In [90]:
IAM_USER = current_user[0].split("@")[0]

# get application default credentials of IAM user (current logged in user)
credentials, project = google.auth.default()

# refresh credentials if expired
if not credentials.valid:
      request = Request()
      credentials.refresh(request)

In [91]:
with pool.connect() as db_conn:
    results = db_conn.execute('show tables;').fetchall()
    
    for row in results: 
        print(row)
    

('Family',)
('Membership',)
('SubscriptionService',)
('University',)
('User',)


In [58]:
fake = Faker()

In [59]:
fake.name()

'David Davis'

In [26]:
from unicodedata import unidata_version

with pool.connect() as con: 
    insert_statement = sqlalchemy.text(
        'insert into University (universityID, universityName, city) values (:universityID, :universityName, :city)'
    )

    con.execute(insert_statement, 
        universityID=0, 
        universityName='University of Illinois at Urbana-Champaign', 
        city='Champaign, IL'
        )
    
    con.execute(insert_statement, 
        universityID=1, 
        universityName='Harvard University',
        city='Cambridge, MA'
        )

    con.execute(insert_statement, 
        universityID=2,
        universityName='Harper College',
        city='Palatine, IL'
        )

    con.execute(insert_statement, 
        universityID=3, 
        universityName='Northwestern University', 
        city='Evanston, IL'
        )

    con.execute(insert_statement, 
        universityID=4, 
        universityName='Massachusetts Institute of Technology', 
        city='Cambridge, MA'
        )

    con.execute(insert_statement, 
        universityID=5, 
        universityName='Yale University',
        city='New Haven, CT'
        )

In [60]:
with pool.connect() as con: 
    results = con.execute('select * from University;').fetchall()
    print(results)

[(0, 'University of Illinois at Urbana-Champaign', 'Champaign, IL'), (1, 'Harvard University', 'Cambridge, MA'), (2, 'Harper College', 'Palatine, IL'), (3, 'Northwestern University', 'Evanston, IL'), (4, 'Massachusetts Institute of Technology', 'Cambridge, MA'), (5, 'Yale University', 'New Haven, CT')]


In [61]:
fake.first_name()
fake.last_name()

'Jones'

In [63]:
with pool.connect() as con: 
    con.execute('delete from User;')

In [67]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into User (userID, userName, email, universityID) values (:userID, :userName, :email, :universityID);'
    )

    for i in range(3001,5000): 
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = first_name + '.' + last_name + '@example.com'
        con.execute(statement, userID=i, userName=first_name + ' ' + last_name, email=email, universityID=random.randint(0, 5))

In [65]:
# with pool.connect() as con: 
#     con.execute('delete from SubscriptionService where serviceName = "Netflix";')

In [47]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into SubscriptionService (serviceName, price, maxMembers) values (:serviceName, :price, :maxMembers);'
    )

    con.execute(statement, serviceName='Netflix', price=19.99, maxMembers=4)
    con.execute(statement, serviceName='Spotify', price=15.99, maxMembers=6)
    con.execute(statement, serviceName='Disney+', price=19.99, maxMembers=4)
    con.execute(statement, serviceName='Amazon Prime Video', price=14.99, maxMembers=3)
    con.execute(statement, serviceName='Hulu', price=12.99, maxMembers=2)

In [66]:
with pool.connect() as con: 
    results = con.execute('select * from SubscriptionService;').fetchall()
    print(results)

[('Amazon Prime Video', Decimal('14.99'), 3), ('Disney+', Decimal('19.99'), 4), ('Hulu', Decimal('12.99'), 2), ('Netflix', Decimal('19.99'), 4), ('Spotify', Decimal('15.99'), 6)]


In [86]:
services = ['Netflix', 'Spotify', 'Disney+', 'Amazon Prime Video', 'Hulu']
n_families = 1000
n_users = 5000
n_members = 3000
userIDs = [i for i in range(n_users)]
familyIDs = [i for i in range(n_families)]
leaderIDs = random.choices(userIDs, k=n_families)#.sample(userIDs, n_families)
accessTypes = ['Open' if random.random() < 0.7 else 'Closed' for _ in range(n_families)]
serviceNames = [services[random.randint(0, 4)] for _ in range(n_families)]

memberIDs = [i for i in range(n_members)]

statuses = ['Accepted', 'Pending', 'Rejected']
member_statuses = random.choices(statuses, weights=[6, 1, 1], k=n_members)

memberships = []
# Do all leader memberships
for i in range(len(leaderIDs)): 
    memberships.append((leaderIDs[i], familyIDs[i], 'Accepted'))

# Do all other memberships
for i in range(n_members): 
    userID = userIDs[random.randint(0, len(userIDs) - 1)]
    familyID = familyIDs[random.randint(0, len(familyIDs) - 1)]
    memberships.append((userID, familyID, member_statuses[i]))

In [87]:
len(set(memberships))

4000

In [98]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into Family (familyID, leaderID, accessType, serviceName) values (:familyID, :leaderID, :accessType, :serviceName);'
    )

    for i in range(len(familyIDs)): 
        con.execute(statement, familyID=familyIDs[i], leaderID=leaderIDs[i], accessType=accessTypes[i], serviceName=serviceNames[i])


In [99]:
with pool.connect() as con: 
    statement = sqlalchemy.text(
        'insert into Membership (memberID, familyID, memberStatus) values (:memberID, :familyID, :memberStatus);'
    )

    for membership in memberships: 
        con.execute(statement, memberID=membership[0], familyID=membership[1], memberStatus=membership[2])


In [105]:
with pool.connect() as con: 
    result = con.execute('select count(*) from Family;').fetchall()
    print(result)

[(1000,)]
